In [1]:
# домашняя работа 

In [97]:
# 1. Теперь решаем задачу регрессии - предскажем цены на недвижимость. 
#    Использовать датасет 
#    https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
# 2. Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию 
#    для оценки качества моделей
# 3. Построить случайный лес, вывести важность признаков
# 4. Обучить стекинг как минимум 3х моделей, использовать 
#    хотя бы 1 линейную модель и 1 нелинейную
# 5. Для валидации модели 2-го уровня использовать 
#    отдельный hold-out датасет, как на занятии
# 6. Показать, что использование ансамблей моделей действительно 
#    улучшает качество (стекинг vs другие модели сравнивать на hold-out)
# 7. В качестве решения: Jupyter notebook с кодом, комментариями 
#    и графиками, ссылка на гитхаб


In [1]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import train_test_split

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('train.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [5]:
# Заполнить средними значениеми
def put_mean_to_na_param(dataset, list_columns):
    for i in list_columns:
        dataset[i].fillna(dataset[i].mean(), inplace=True)

In [6]:
# Заполнить значением
def put_value_to_na_param(dataset, list_columns, value=0 ):
    for i in list_columns:
        dataset[i].fillna(value, inplace=True)

In [7]:
# Заполнить самым распространенным значением
def put_most_common_meaning_to_na_param(dataset, list_columns):
    for i in list_columns:
        dataset[i].fillna(dataset[i].value_counts()[0], inplace=True)

In [8]:
# Заполнить значения NaN значениеями из другой колонки
def put_param_from_col_src_to_na_param(dataset, col_dest, col_src): 
    dataset[col_dest]=dataset.apply(lambda x: x[col_src] if x[col_dest] is not None else x[col_dest], axis=1 )
            

In [9]:
# Закодировать nominal категориальную переменную с помощью "one hot encoding"  (используем реализацию из лекций)
def get_one_hot(df, cols):
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    return df 

# Закодировать ordered категориальную переменную используя числовой порядок
def get_ordered_col(dataset, col, dict_col_params):
    """
    В функцию передается датасет, имя колонки и словарь {'значение_параметра_в_колонке': числовое_значение_для_замены }
    """
    dataset[col + '_order'] = dataset.apply(lambda x: dict_col_params[x[col]],axis=1)
    return dataset

def get_ordered_cols(dataset, cols, dict_col_params):
    for col in cols:
        res = get_ordered_col(dataset, col, dict_col_params)
    return res.drop(columns=cols)     

dict_label_5_1 = {'Ex': 4,'Gd': 3,'TA': 2,'Fa': 1,'Po': 0}
dict_label_5_2 = {'Ex': 4,'Gd': 3,'TA': 2,'Fa': 1,'NA': 0}
dict_label_5_3 = {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'NA': 0} 
dict_label_6_1 = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}
dict_label_7_1 = {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'NA': 0} 


In [10]:
# функция подготовки данных
def get_data(dataset):
    put_value_to_na_param(dataset, ['MiscFeature','Alley','Fence','BsmtQual','PoolQC','BsmtCond',\
                                    'BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu',\
                                    'GarageType','GarageFinish','GarageQual','GarageCond'], 'NA' )
    
    put_value_to_na_param(dataset, ['MasVnrArea'], 0)
    put_value_to_na_param(dataset, ['MasVnrType'], 'none')                      
    put_mean_to_na_param(dataset, ['LotFrontage'])
    put_most_common_meaning_to_na_param(dataset, ['Electrical'])   
    put_param_from_col_src_to_na_param(dataset, 'GarageYrBlt', 'YearRemodAdd')     
    
    data_raw = dataset.copy()
    data_raw = get_one_hot(dataset, ['MiscFeature', 'Fence', 'Alley', 'Street', 'LotShape', 'LandContour', 'Utilities',\
                                     'LotConfig', 'LandSlope', 'BldgType', 'MasVnrType', 'Foundation', 'Heating', \
                                     'CentralAir', 'Electrical', 'PavedDrive', 'MSZoning','MSSubClass','Neighborhood','Condition1',\
                                     'Condition2','HouseStyle','RoofStyle','RoofMatl','Exterior1st',\
                                     'Exterior2nd','Functional','GarageType','GarageFinish',\
                                     'SaleType','SaleCondition'])
    
    data_raw = get_ordered_cols(data_raw, ['ExterQual', 'ExterCond', 'HeatingQC', 'KitchenQual'], dict_label_5_1)
    data_raw = get_ordered_cols(data_raw, ['PoolQC'], dict_label_5_2)
    data_raw = get_ordered_cols(data_raw, ['BsmtExposure'], dict_label_5_3)
    data_raw = get_ordered_cols(data_raw, ['BsmtQual', 'BsmtCond', 'FireplaceQu', 'GarageQual', 'GarageCond'] , dict_label_6_1)
    data_raw = get_ordered_cols(data_raw, ['BsmtFinType1', 'BsmtFinType2'] , dict_label_7_1) 
    
    y = data_raw['SalePrice'].copy()
    #data_raw= data_raw.drop(columns=['SaleCategory'])
    columns_to_drop = ['SalePrice', 'MSZoning','MSSubClass','Neighborhood','Condition1',\
                       'Condition2','HouseStyle','RoofStyle','RoofMatl','Exterior1st',\
                       'Exterior2nd','Functional','GarageType','GarageFinish',\
                       'SaleType','SaleCondition','MiscFeature', 'Fence', 'Alley',\
                       'Street', 'LotShape', 'LandContour', 'Utilities',\
                       'LotConfig', 'LandSlope', 'BldgType', 'MasVnrType', \
                       'Foundation', 'Heating', 'CentralAir', 'Electrical', 'PavedDrive']
    
    data_raw = data_raw.drop(columns=columns_to_drop)
    
        
    
    return data_raw, y
    

In [11]:
X_full, y_full = get_data(data)

In [12]:
X_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 263 entries, Id to BsmtFinType2_order
dtypes: float64(2), int64(47), uint8(214)
memory usage: 864.1 KB


In [13]:
X_full.head()

,Id,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,KitchenQual_order,PoolQC_order,BsmtExposure_order,BsmtQual_order,BsmtCond_order,FireplaceQu_order,GarageQual_order,GarageCond_order,BsmtFinType1_order,BsmtFinType2_order
0,1,65.0,8450,7,5,2003,2003,196.0,706,0,...,3,0,1,4,3,0,3,3,6,1
1,2,80.0,9600,6,8,1976,1976,0.0,978,0,...,2,0,4,4,3,3,3,3,5,1
2,3,68.0,11250,7,5,2001,2002,162.0,486,0,...,3,0,2,4,3,3,3,3,6,1
3,4,60.0,9550,7,5,1915,1970,0.0,216,0,...,3,0,1,3,4,4,3,3,5,1
4,5,84.0,14260,8,5,2000,2000,350.0,655,0,...,3,0,3,4,3,3,3,3,6,1


In [14]:
# X и y общая выборка для моделей 1 уровня 
# X_out,y_out - выборка для проверки модели второго уровня

X, X_out, y, y_out = train_test_split(X_full.drop(columns=['Id']), y_full.values, test_size=0.1 ,random_state=42)

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
sc = StandardScaler().fit(X)
X_keys = X.columns
X = sc.transform(X)
X_out = sc.transform(X_out)
X = pd.DataFrame(X, columns=X_keys)
X_out = pd.DataFrame(X_out, columns=X_keys)

In [17]:
# 3. Построить случайный лес, вывести важность признаков

from sklearn.ensemble import RandomForestRegressor

In [26]:
reg = RandomForestRegressor()
reg.fit(X,y)
print(reg.score(X,y))

feature_importances= {}
for col, val in zip(X.columns, reg.feature_importances_):
    feature_importances[col] = val
    
sorted_x = sorted(feature_importances.items(), key=lambda kv: kv[1], reverse=True)
sorted_x[:20]

0.9757689406667408


[('OverallQual', 0.5264482678244882),
 ('GrLivArea', 0.11250606766573341),
 ('GarageCars', 0.050224938049695925),
 ('TotalBsmtSF', 0.049715846687454074),
 ('2ndFlrSF', 0.041714517808706074),
 ('BsmtFinSF1', 0.024674303855184453),
 ('1stFlrSF', 0.02258822823839033),
 ('YearBuilt', 0.013063641255038916),
 ('LotArea', 0.011605307725973344),
 ('FullBath', 0.011296898344758447),
 ('GarageArea', 0.010431978703686859),
 ('FireplaceQu_order', 0.00970786414634318),
 ('LotFrontage', 0.008672064673418337),
 ('KitchenQual_order', 0.008152220274634454),
 ('BsmtQual_order', 0.005282337492803041),
 ('BsmtUnfSF', 0.005016187633810574),
 ('MasVnrArea', 0.00417066714873981),
 ('GarageYrBlt', 0.003913792943300662),
 ('OpenPorchSF', 0.0038467869649789627),
 ('MoSold', 0.0036614479329417937)]

In [27]:
# 4. Обучить стекинг как минимум 3х моделей, использовать 
#    хотя бы 1 линейную модель и 1 нелинейную
# 5. Для валидации модели 2-го уровня использовать 
#    отдельный hold-out датасет, как на занятии

In [28]:
# Для стекинга используем следующие типы моделей
# модели первого уровня
# 1. SGDRegressor
# 2. Ridge
# 3. KNeighborsRegressor
# модель второго уровня
# 1. LinearRegression

In [29]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ARDRegression

from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [68]:
def get_MAX(data):
    data_max = data.copy()
    keys = data_max.keys()
    for key1 in keys:
        for key2 in keys:
            data_max[key1+'_x_'+key2] = data_max[key1]*data_max[key2]
    return data_max.drop(columns=keys)

In [69]:
def make_stacking(X, y, dict_model, meta_model, param_model_GS, param_meta_model, make_max = False):
    # получаем длину фолда
    len_fold = (len(X) // len(dict_model.keys()))+1
    kf = KFold(n_splits=len(dict_model.keys()), shuffle=True)
    result_fold = {}
    i = 0
    for train_index, test_index in kf.split(X):
        test_index_l = list(test_index)
        train_index_l =  list(train_index)
        # для того что бы данные для мета модели были полные
        # выравниваем длину фолдов
        # переносим избыток индексов из train в test
        if(len_fold > len(test_index)):
            # получим номера индексов в train  для переноса в test   
            add_num = random.sample(range(len(train_index)), len_fold > len(test_index))
            # получим индексы из train
            list_change_elem = train_index[add_num]
            # пройдем по списку индексов 
            for k in list_change_elem:
                # удалим индекс из train
                train_index_l.remove(k)
                #  добавим индекс в test
                test_index_l = test_index_l + list([k,])
        # -----------------------        
        # формируем выборки данных для фолдов
        k = list(dict_model.keys())[i]
        result_fold[k] = {'X_train': X.iloc[train_index_l], 'X_test': X.iloc[test_index_l],
                          'y_train': y[train_index_l], 'y_test': y[test_index_l]}
        # ------------------------
        i+=1
    result_model ={}
    # проходим по списку моделей 1 уровня 
    print('stacking')
    for j in dict_model.keys():
        # обучаем 
        dict_model[j] = GridSearchCV(dict_model[j], param_model_GS[j], cv=5, n_jobs=-1).fit(result_fold[j]['X_train'], result_fold[j]['y_train']).best_estimator_
        # предсказания помещаем в словарь с данными для мета модели
        result_model[j] = dict_model[j].predict(result_fold[j]['X_test'])
        print('MAE ', j, ":",mean_absolute_error(result_model[j], result_fold[j]['y_test']))
    df_result = pd.DataFrame.from_dict(result_model)
    # обучаем мета модел
    df_result_2 = get_MAX(df_result) if make_max else df_result
    meta_model_best =  GridSearchCV(meta_model, param_meta_model, cv=5, n_jobs=-1).fit(df_result_2,result_fold[j]['y_test']).best_estimator_
    print('MAE stacking:',mean_absolute_error(meta_model_best.predict(df_result_2), result_fold[j]['y_test']))
    return dict_model, meta_model_best

In [74]:
def make_blanding(X, y, dict_model, meta_model,param_model_GS, param_meta_model, make_max = False):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)
    result_model ={}
    print('blanding')
    for i in dict_model.keys():
        dict_model[i] = GridSearchCV(dict_model[i], param_model_GS[i], cv=5, n_jobs=-1).fit(X_train, y_train).best_estimator_
        result_model[i] = dict_model[i].predict(X_test)
        print('MAE ', i, ":",mean_absolute_error(result_model[i], y_test))
    df_result = pd.DataFrame.from_dict(result_model)
    df_result_2 = get_MAX(df_result) if make_max else df_result
    meta_model_best = GridSearchCV(meta_model, param_meta_model, cv=5, n_jobs=-1).fit(df_result_2,y_test).best_estimator_
    print('MAE blanding:',mean_absolute_error(meta_model_best.predict(df_result_2), y_test))
    return dict_model, meta_model_best

In [71]:
def predict_result(X, dict_model, meta_model, make_max = False ):
    result_model ={}
    for i in dict_model.keys():
        result_model[i] = dict_model[i].predict(X)
    df_result = pd.DataFrame.from_dict(result_model)
    df_result_2 = get_MAX(df_result) if make_max else df_result
    result = meta_model.predict(df_result_2) 
    return result

In [42]:
param_model_GS = {'Ridge':{'alpha':[0.1,0.5,1]}, 
              'Lasso':{'alpha':[0.1,0.5,1]}, 
              'KNeighborsRegressor':{'n_neighbors':[3,5,7,9]},
              'LinearSVR':{'C':[0.1,0.5,1], 'loss':['epsilon_insensitive', 'squared_epsilon_insensitive']},
              'DecisionTreeRegressor':{'max_depth':[3,5,7,9]},                                    
              'LinearRegression':{},
              'SGDRegressor':{'loss':['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
                              'penalty':['none', 'l2', 'l1','elasticnet'] 
                             },
              'ARDRegression':{'alpha_1':[0.0000001,0.000001,0.00001,],'lambda_1':[0.0000001,0.000001,0.00001,]}   
             }



dict_model = {}
dict_model['KNeighborsRegressor'] = KNeighborsRegressor()
dict_model['Ridge'] = Ridge()
dict_model['Lasso'] = Lasso()
dict_model['LinearSVR'] = LinearSVR()
#dict_model['SGDRegressor'] = SGDRegressor()
dict_model['DecisionTreeRegressor'] = DecisionTreeRegressor()
#dict_model['LinearRegression'] = LinearRegression()
meta_model = ARDRegression()


blending_dict_model, blending_meta_model = make_blanding(X, y, dict_model, meta_model,param_model_GS, param_model_GS['ARDRegression'])
blending_result = predict_result(X_out, blending_dict_model, blending_meta_model)

stacking_dict_model, stacking_meta_model = make_stacking(X, y, dict_model, meta_model,param_model_GS, param_model_GS['ARDRegression'])
stacking_result = predict_result(X_out, stacking_dict_model, stacking_meta_model)




blanding
MAE  KNeighborsRegressor : 26231.96600361664
MAE  Ridge : 19903.10481211439
MAE  Lasso : 19791.901001248203
MAE  LinearSVR : 23783.662911215048
MAE  DecisionTreeRegressor : 28118.022873764745
MAE blanding: 18045.44571364402
stacking
MAE  KNeighborsRegressor : 23855.769328263625
MAE  Ridge : 17539.18507562447
MAE  Lasso : 19954.035711675737
MAE  LinearSVR : 24783.966184263645
MAE  DecisionTreeRegressor : 27476.722034030103
MAE stacking: 27525.572649960097


In [43]:
blending_dict_model

{'DecisionTreeRegressor': DecisionTreeRegressor(criterion='mse', max_depth=9, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
 'KNeighborsRegressor': KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=9, p=2,
           weights='uniform'),
 'Lasso': Lasso(alpha=1, copy_X=True, fit_intercept=True, max_iter=1000,
    normalize=False, positive=False, precompute=False, random_state=None,
    selection='cyclic', tol=0.0001, warm_start=False),
 'LinearSVR': LinearSVR(C=0.1, dual=True, epsilon=0.0, fit_intercept=True,
      intercept_scaling=1.0, loss='squared_epsilon_insensitive',
      max_iter=1000, random_state=None, tol=0.0001, verbose=0),
 'Ridge': Ridge(alpha=1, copy_X=True, fit_intercept=True, max_ite

In [44]:
# сравнение моделей
# для сравнения используем все простые модели используемые в ансамблировании
# и ансамблевые модели представленные в sklearn

In [45]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor

param_model_GS['RandomForestRegressor'] = {'max_depth':[3,5,7,9]}
param_model_GS['BaggingRegressor'] = {'base_estimator':[Ridge(), Lasso()] }
param_model_GS['GradientBoostingRegressor'] = {'loss' : ['ls', 'lad', 'huber', 'quantile']}
param_model_GS['AdaBoostRegressor'] = {'loss' : ['linear', 'square', 'exponential'],'base_estimator':[Ridge(), Lasso(), None] }

In [46]:
Rg  = GridSearchCV(Ridge(), param_model_GS['Ridge'], cv=5, n_jobs=-1).fit(X,y).best_estimator_.predict(X_out)
Ls  = GridSearchCV(Lasso(), param_model_GS['Lasso'], cv=5, n_jobs=-1).fit(X,y).best_estimator_.predict(X_out)
KNR = GridSearchCV(KNeighborsRegressor(), param_model_GS['KNeighborsRegressor'], cv=5, n_jobs=-1).fit(X,y).best_estimator_.predict(X_out)
SVR = GridSearchCV(LinearSVR(), param_model_GS['LinearSVR'], cv=5, n_jobs=-1).fit(X,y).best_estimator_.predict(X_out)
DTR = GridSearchCV(DecisionTreeRegressor(), param_model_GS['DecisionTreeRegressor'], cv=5, n_jobs=-1).fit(X,y).best_estimator_.predict(X_out)
SGD = GridSearchCV(SGDRegressor(), param_model_GS['SGDRegressor'], cv=5, n_jobs=-1).fit(X,y).best_estimator_.predict(X_out)
LR  = LinearRegression().fit(X,y).predict(X_out)

In [47]:
RFR = GridSearchCV(RandomForestRegressor(), param_model_GS['RandomForestRegressor'], cv=5, n_jobs=-1).fit(X,y).best_estimator_.predict(X_out)
BR  = GridSearchCV(BaggingRegressor(), param_model_GS['BaggingRegressor'], cv=5, n_jobs=-1).fit(X,y).best_estimator_.predict(X_out)
GBR = GridSearchCV(GradientBoostingRegressor(), param_model_GS['GradientBoostingRegressor'], cv=5, n_jobs=-1).fit(X,y).best_estimator_.predict(X_out)
ABR = GridSearchCV(AdaBoostRegressor(), param_model_GS['AdaBoostRegressor'], cv=5, n_jobs=-1).fit(X,y).best_estimator_.predict(X_out)

In [48]:
print('blending:', mean_absolute_error(blending_result,y_out))
print('stacking:', mean_absolute_error(stacking_result,y_out))
print('ridge   :', mean_absolute_error(Rg,y_out))
print('lasso   :', mean_absolute_error(Ls,y_out))
print('KNR     :', mean_absolute_error(KNR,y_out))
print('SVR     :', mean_absolute_error(SVR,y_out))
print('DTR     :', mean_absolute_error(DTR,y_out))
print('SGD     :', mean_absolute_error(SGD,y_out))
print('LR      :', mean_absolute_error(LR,y_out))
print('RFR     :', mean_absolute_error(RFR,y_out))
print('BR      :', mean_absolute_error(BR,y_out))
print('GBR     :', mean_absolute_error(GBR,y_out))
print('ABR     :', mean_absolute_error(ABR,y_out))

blending: 20803.113328631985
stacking: 23381.801057183777
ridge   : 20333.968734261707
lasso   : 20514.189099687002
KNR     : 27300.959665144597
SVR     : 20280.540700349447
DTR     : 20104.89371316692
SGD     : 182859.6699776689
LR      : 1284451250444319.5
RFR     : 18094.231185598113
BR      : 19598.48365304687
GBR     : 15241.677938146157
ABR     : 22901.66109759587


In [49]:
print('blending:', r2_score(blending_result,y_out))
print('stacking:', r2_score(stacking_result,y_out))
print('ridge   :', r2_score(Rg,y_out))
print('lasso   :', r2_score(Ls,y_out))
print('KNR     :', r2_score(KNR,y_out))
print('SVR     :', r2_score(SVR,y_out))
print('DTR     :', r2_score(DTR,y_out))
print('SGD     :', r2_score(SGD,y_out))
print('LR      :', r2_score(LR,y_out))
print('RFR     :', r2_score(RFR,y_out))
print('BR      :', r2_score(BR,y_out))
print('GBR     :', r2_score(GBR,y_out))
print('ABR     :', r2_score(ABR,y_out))

blending: 0.842182885301606
stacking: 0.8384602064042123
ridge   : 0.8605925614126352
lasso   : 0.8604944146358682
KNR     : 0.2301631689014455
SVR     : 0.8596783338475624
DTR     : 0.8692136696723076
SGD     : -143266804255.90283
LR      : -0.0005560992373536688
RFR     : 0.8149751394507361
BR      : 0.8618133837446534
GBR     : 0.8853853928363336
ABR     : 0.7813089745523257


In [50]:
# Сравнение моделей показало, # что такая реализация не увеличивает 
# качество ответа по сравнению с некотрыми обычными и ансамблевыми моделями

# 

In [51]:
# Попробуем различные методики повышения качества стекинга
#
# 1. Следуя советам данным в https://dyakonov.org/2017/03/10/cтекинг-stacking-и-блендинг-blending 
#   а) не будем подбирать для моделей гиперпараметры, 
#      а будем использовать несколько модели одного типа с разным их набором
#   б) мета модель будем обучать не на непосредственно полученых признаках,   
#      а на сгенерированных с помощью их декартова произведения  
#
#  2. В качестве моделей возмем 
#    а) Ridge
#    б) Lasso
#    б) DecisionTreeRegressor
#    б) SVR
#    б) KNR
#  Каждую из моделей возмем 3-5 вариантах по гиперпараметрам
#  В качестве мета модели возмем Ridge 

In [72]:
from sklearn.svm import SVR

param_model_GS_2={'Ridge1':{'alpha':[0.01]},
                  'Ridge2':{'alpha':[0.05]}, 
                  'Ridge3':{'alpha':[0.1]}, 
                  'Ridge4':{'alpha':[0.5]}, 
                  'Ridge5':{'alpha':[1]},
                  'Ridge5':{'alpha':[5]},
                  'Lasso1':{'alpha':[0.01]}, 
                  'Lasso2':{'alpha':[0.05]}, 
                  'Lasso3':{'alpha':[0.1]}, 
                  'Lasso4':{'alpha':[0.5]}, 
                  'Lasso5':{'alpha':[1]},
                  'Lasso6':{'alpha':[5]},
    'KNeighborsRegressor1':{'n_neighbors':[3]},
    'KNeighborsRegressor2':{'n_neighbors':[5]},
    'KNeighborsRegressor3':{'n_neighbors':[7]},
    'KNeighborsRegressor4':{'n_neighbors':[9]},
                    'SVR1':{'kernel':['linear'], 'C':[0.1,0.5,1]},
                    'SVR2':{'kernel':['poly'], 'C':[0.1,0.5,1]},
                    'SVR3':{'kernel':['rbf'], 'C':[0.1,0.5,1]},
                    'SVR4':{'kernel':['sigmoid'], 'C':[0.1,0.5,1]},
  'DecisionTreeRegressor1':{'max_depth':[1]},                                   
  'DecisionTreeRegressor2':{'max_depth':[3]},                                    
  'DecisionTreeRegressor3':{'max_depth':[5]},                                    
  'DecisionTreeRegressor4':{'max_depth':[7]},                                    
  'DecisionTreeRegressor5':{'max_depth':[9]},                                    
             }
dict_model_2 = {}
dict_model_2['Ridge1'] = Ridge()
dict_model_2['Ridge2'] = Ridge()
dict_model_2['Ridge3'] = Ridge()
dict_model_2['Ridge4'] = Ridge()
dict_model_2['Ridge5'] = Ridge()
dict_model_2['Ridge5'] = Ridge()
dict_model_2['Lasso1'] = Lasso()
dict_model_2['Lasso2'] = Lasso()
dict_model_2['Lasso3'] = Lasso()
dict_model_2['Lasso4'] = Lasso()
dict_model_2['Lasso5'] = Lasso()
dict_model_2['Lasso6'] = Lasso()
dict_model_2['KNeighborsRegressor1'] = KNeighborsRegressor()
dict_model_2['KNeighborsRegressor2'] = KNeighborsRegressor()
dict_model_2['KNeighborsRegressor3'] = KNeighborsRegressor()
dict_model_2['KNeighborsRegressor4'] = KNeighborsRegressor()
dict_model_2['SVR1'] = SVR()
dict_model_2['SVR2'] = SVR()
dict_model_2['SVR3'] = SVR()
dict_model_2['SVR4'] = SVR()
dict_model_2['DecisionTreeRegressor1'] = DecisionTreeRegressor()
dict_model_2['DecisionTreeRegressor2'] = DecisionTreeRegressor()
dict_model_2['DecisionTreeRegressor3'] = DecisionTreeRegressor()
dict_model_2['DecisionTreeRegressor4'] = DecisionTreeRegressor()
dict_model_2['DecisionTreeRegressor5'] = DecisionTreeRegressor()



In [81]:
blending_dict_model_2, blending_meta_model_2 = make_blanding(X, y, dict_model_2, Ridge(),param_model_GS_2, {'alpha':[0.5,1,5]})
blending_result_2 = predict_result(X_out, blending_dict_model_2, blending_meta_model_2)

stacking_dict_model_2, stacking_meta_model_2 = make_stacking(X, y, dict_model_2, Ridge(),param_model_GS_2, {'alpha':[0.5,1,5]})   
stacking_result_2 = predict_result(X_out, stacking_dict_model_2, stacking_meta_model_2)

blanding
MAE  Ridge1 : 21610.447144125865
MAE  Ridge2 : 21604.463602500993
MAE  Ridge3 : 21597.034455341767
MAE  Ridge4 : 21539.507626585633
MAE  Ridge5 : 21073.325462342225
MAE  Lasso1 : 21999.61337348825
MAE  Lasso2 : 21974.07288843056
MAE  Lasso3 : 21950.02916311487
MAE  Lasso4 : 21906.54937257303
MAE  Lasso5 : 21854.56904458646
MAE  Lasso6 : 21492.68443049822
MAE  KNeighborsRegressor1 : 27078.34936708861
MAE  KNeighborsRegressor2 : 26765.514936708863
MAE  KNeighborsRegressor3 : 26182.174321880648
MAE  KNeighborsRegressor4 : 26199.772995780586
MAE  SVR1 : 48262.09987057522
MAE  SVR2 : 56895.90039880984
MAE  SVR3 : 56880.830365204754
MAE  SVR4 : 56863.805782192416
MAE  DecisionTreeRegressor1 : 44158.596566529675
MAE  DecisionTreeRegressor2 : 33314.06119291947
MAE  DecisionTreeRegressor3 : 30190.99766550298
MAE  DecisionTreeRegressor4 : 29266.692853775145
MAE  DecisionTreeRegressor5 : 27208.11259415961
MAE blanding: 18261.45645771932
stacking
MAE  Ridge1 : 16098.896629925803
MAE  Ridg

In [82]:
print('blending:', mean_absolute_error(blending_result_2,y_out))
print('stacking:', mean_absolute_error(stacking_result_2,y_out))

print('blending:', r2_score(blending_result_2,y_out))
print('stacking:', r2_score(stacking_result_2,y_out))

blending: 21888.07941495229
stacking: 36383.55567960053
blending: 0.798548476197396
stacking: -0.1822211349742895


In [83]:
blending_dict_model_3, blending_meta_model_3 = make_blanding(X, y, dict_model_2, Ridge(),param_model_GS_2, {'alpha':[0.5,1,5]}, True)
blending_result_3 = predict_result(X_out, blending_dict_model_3, blending_meta_model_3, True)

stacking_dict_model_3, stacking_meta_model_3 = make_stacking(X, y, dict_model_2, Ridge(),param_model_GS_2, {'alpha':[0.5,1,5]}, True)   
stacking_result_3 = predict_result(X_out, stacking_dict_model_3, stacking_meta_model_3, True)

blanding
MAE  Ridge1 : 20517.52749657536
MAE  Ridge2 : 20514.66684839883
MAE  Ridge3 : 20511.107933967727
MAE  Ridge4 : 20484.420429224323
MAE  Ridge5 : 20285.145771179956
MAE  Lasso1 : 20564.502451166758
MAE  Lasso2 : 20562.163679281104
MAE  Lasso3 : 20559.383096568752
MAE  Lasso4 : 20537.05537178594
MAE  Lasso5 : 20513.168936926282
MAE  Lasso6 : 20417.90556105136
MAE  KNeighborsRegressor1 : 27018.383966244728
MAE  KNeighborsRegressor2 : 26124.079999999998
MAE  KNeighborsRegressor3 : 25390.640144665464
MAE  KNeighborsRegressor4 : 25456.52995780591
MAE  SVR1 : 44186.08428915427
MAE  SVR2 : 52367.03427606273
MAE  SVR3 : 52352.01670197039
MAE  SVR4 : 52335.28783957428
MAE  DecisionTreeRegressor1 : 42764.69752638832
MAE  DecisionTreeRegressor2 : 33060.7787770497
MAE  DecisionTreeRegressor3 : 27405.38293513473
MAE  DecisionTreeRegressor4 : 26743.788224366202
MAE  DecisionTreeRegressor5 : 26857.75786993509
MAE blanding: 8701.36595620596
stacking
MAE  Ridge1 : 19650.247164899174
MAE  Ridge2 

In [84]:
print('blending:', mean_absolute_error(blending_result_3,y_out))
print('stacking:', mean_absolute_error(stacking_result_3,y_out))

print('blending:', r2_score(blending_result_3,y_out))
print('stacking:', r2_score(stacking_result_3,y_out))

blending: 329082.98877160146
stacking: 25527.902440918213
blending: -0.0037908491608562578
stacking: 0.8856849860628366


In [ ]:
# судя по результатам можно сделать следующие наблюдения
# а) для случая без генерации дополнительных переменных блендинг и стекинг отработали хуже
#    наблюдается переобучение 
# б) для случая с генерацией дополнительных переменных стекинг отработал лучше, 
#    а блендинг еще сильней переобучился 